In [23]:
import pandas as pd
import numpy as np

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import VarianceThreshold



from cp_app.utils import select_structures
from cp_app.descriptors import cv_features

import warnings
warnings.filterwarnings('ignore')

In [33]:
FEATURES = cv_features
TARGET = 'pCv_300.00' 

In [34]:
def xgb_model(best_params, X, FEATURES, TARGET):
    pipe_xgb = Pipeline(
        [
       ('scaling', StandardScaler()), 
       ('variance_threshold', VarianceThreshold(threshold=0.95)),
       ('xgb', XGBRegressor())
       
       ]
    )
    pipe_xgb.set_params(xgb__reg_lambda = best_params["xgb__reg_lambda"])
    pipe_xgb.set_params(xgb__reg_alpha = best_params["xgb__reg_alpha"])
    pipe_xgb.set_params(xgb__n_estimators = best_params["xgb__n_estimators"])
    pipe_xgb.set_params(xgb__max_depth = best_params["xgb__max_depth"])
    pipe_xgb.set_params(xgb__learning_rate = best_params["xgb__learning_rate"])
    
    pipe_xgb.fit(X[FEATURES], X[TARGET])
    
    return pipe_xgb
    

## Data loading, and train test split,...

In [29]:
RANDOM_SEED = 3982
DATA_DIR = 'data/'
DATA_site = 'alldata.csv'
DATA_structure = 'structures_data.csv'
flag="smallML_120_5" # name_#TrainSize_#EnsembleSize 
N_ensemble = int(flag.split("_")[2])
N_train=int(flag.split("_")[1])

In [30]:
df_structures=pd.read_csv(DATA_DIR+DATA_structure)
df_allsites = pd.read_csv(DATA_DIR+DATA_site)
print("Total structures: ", len(df_structures))

Total structures:  232


### train test split, making sure we have diverse training set

In [32]:
inds=select_structures(N_train,df_structures)
df_train_structures=df_structures.loc[df_structures.index.isin(inds)]
df_test_structures=df_structures.loc[~df_structures.index.isin(inds)]
df_train_structures.to_csv(DATA_DIR+"train_structures.csv")
df_test_structures.to_csv(DATA_DIR+"test_structures.csv")
print("{} structures in training set".format(len(df_train_structures)))
print("{} structures in test set".format(len(df_test_structures)))

120 structures in training set
112 structures in test set


In [14]:
df = df_allsites.loc[df_allsites["structure_name"].isin(df_train_structures["Unnamed: 0"])]
df_site_test = df_allsites.loc[~df_allsites["structure_name"].isin(df_train_structures["Unnamed: 0"])]
df.to_csv(DATA_DIR+"site_train.csv",index=False)

In [18]:
len(df["structure_name"].unique())

120

In [2]:
cif_list = glob.glob("cifs/*")

In [3]:
df_features = featurize_dataset(cif_list, verbos=False, saveto="data/features_example1.csv")

/Users/mohamad/anaconda3/envs/cp_env/lib/python3.8/site-packages/pymatgen/io/cif.py:1160: UserWarning: Issues encountered while parsing CIF: Some fractional coordinates rounded to ideal values to avoid issues with finite precision.
  warnings.warn("Issues encountered while parsing CIF: %s" % "\n".join(self.warnings))


site elemental properties
AGNI
GSF
LPD


In [4]:
df_features.head()

,site Number,site AtomicWeight,site Row,site Column,site Electronegativity,site CovalentRadius,AGNI eta=8.00e-01,AGNI eta=1.23e+00,AGNI eta=1.88e+00,AGNI eta=2.89e+00,...,maximum local difference in Row,maximum local difference in Column,maximum local difference in Electronegativity,maximum local difference in CovalentRadius,minimum local difference in Number,minimum local difference in AtomicWeight,minimum local difference in Row,minimum local difference in Column,minimum local difference in Electronegativity,minimum local difference in CovalentRadius
cifs/18150N2_0,1.0,1.00794,1.0,1.0,2.2,31.0,0.139803,0.467199,1.157492,2.414420,...,2.0,16.0,1.24,71.0,0.0,0.0,0.0,0.0,0.0,0.0
cifs/18150N2_1,1.0,1.00794,1.0,1.0,2.2,31.0,0.130159,0.529070,1.725751,3.807856,...,1.0,15.0,1.24,45.0,0.0,0.0,0.0,0.0,0.0,0.0
cifs/18150N2_2,1.0,1.00794,1.0,1.0,2.2,31.0,0.141849,0.513710,1.523709,3.365105,...,2.0,16.0,1.24,71.0,0.0,0.0,0.0,0.0,0.0,0.0
cifs/18150N2_3,1.0,1.00794,1.0,1.0,2.2,31.0,0.136132,0.509148,1.518896,3.388150,...,1.0,15.0,1.24,45.0,0.0,0.0,0.0,0.0,0.0,0.0
cifs/18150N2_4,1.0,1.00794,1.0,1.0,2.2,31.0,0.139793,0.498658,1.348067,2.838402,...,2.0,16.0,1.24,71.0,0.0,0.0,0.0,0.0,0.0,0.0
